In [136]:
#the xlrd library is necessary for read xls file  
#%pip install xlrd 

import pandas as pd
import numpy as np
import regex as re
from datetime import datetime
from time import time
from time import sleep
from math import ceil
from datetime import timedelta
import unicodedata  
import os
from IPython.display import clear_output
from difflib import SequenceMatcher 

In [137]:
%cd Data_Source

start_time = time()
#documentating information about the excel files and sheets inside of it.
files = os.listdir()
#getting all the files with xlsx or xls format
xlsx_files = [x for x in files if x.split('.')[-1]=='xlsx' or  x.split('.')[-1]=='xls']

def get_sheets_names(x):
    xlsx = pd.ExcelFile(x)
    return xlsx.sheet_names  # see all sheet names

#putting all the sheets names inside a list
sheets_names = [get_sheets_names(excel) for excel in xlsx_files]

#zipping the lists togueter for execute as program
lst_src = list(zip(xlsx_files, sheets_names))

#building a dataframe with information about the excel files
df_xlsx = pd.DataFrame(lst_src, columns=['xlsx_file','sheets'])

for source in lst_src: print(f'EXCEL: {source[0]} >>> SHEETS: {source[1]}')

%cd ..

/home/roberto/github/projeto_controle_financeiro_gesac-11/Data_Source
EXCEL: Controle de Empenhos e NC 2022.xlsx >>> SHEETS: ['ControleGeral', 'SintéticoGeral', 'SintéticoNC', 'AnalíticoNC', 'ÍndiceCorreçãoNC']
EXCEL: Controle de NC 2022.xlsx >>> SHEETS: ['SintéticoNC', 'AnalíticoNC', 'ÍndiceCorreção']
EXCEL: Emendas2021_resumo.xlsx >>> SHEETS: ['Planilha1']
EXCEL: Orçamentário MCOM 2021 16-11-21 setel.xlsx >>> SHEETS: ['Adm. Direta - Discricionárias', 'Emendas Individuais', 'Emendas de Bancada', 'dinamica', 'base']
EXCEL: Proponente.xlsx >>> SHEETS: ['Sheet1']
EXCEL: Proponentes.xlsx >>> SHEETS: ['Sheet1']
EXCEL: Relatorio_SISGESAC 1-02-2022.xls >>> SHEETS: ['Extração']
EXCEL: estados.xlsx >>> SHEETS: ['estados']
/home/roberto/github/projeto_controle_financeiro_gesac-11


In [138]:
%cd Data_Source

#!!!
df_nm =[
('df_an', 'AnalíticoNC'),
# ('df_ei', 'Emendas Individuais'),
('df_sn', 'SintéticoNC'), 
('df_ic', 'ÍndiceCorreção'),
('df_pr', 'Sheet1'),
('df_cg', 'ControleGeral'),
('df_em', 'Planilha1'),
('df_estados', 'estados'),
('df_sg', 'Extração')] 

df_sheets = pd.DataFrame(df_nm, columns = ['df_name', 'sheet_name'])
df_sheets['xlsx_file']= ''

for i1, sheets in enumerate(df_xlsx['sheets']):
    for i2, sheet in enumerate(df_sheets['sheet_name']):
        if sheet in sheets:
            df_sheets['xlsx_file'][i2] = df_xlsx['xlsx_file'][i1] 

def parameters(df):
    excel = df_sheets.loc[df_sheets.df_name == df]['xlsx_file'].values[0]
    sheet = df_sheets.loc[df_sheets.df_name == df]['sheet_name'].values[0]
    return excel, sheet

xlsx, sheet = parameters('df_an')
df_an = pd.read_excel(xlsx,sheet_name=sheet,
usecols="B:J", skiprows=1,parse_dates=True)

xlsx, sheet = parameters('df_sn')
df_sn = pd.read_excel(xlsx, sheet_name=sheet,
usecols="B:J", skiprows=3,parse_dates=True)

xlsx, sheet = parameters('df_ic')
df_ic = pd.read_excel(xlsx, sheet_name=sheet,
skiprows=1,usecols="B:E")

xlsx, sheet = parameters('df_pr')
df_pr = pd.read_excel(xlsx)

xlsx, sheet = parameters('df_cg')
df_cg = pd.read_excel(xlsx, sheet_name=sheet
, skiprows=12,usecols="c:w")

xlsx, sheet = parameters('df_em')
df_em= pd.read_excel(xlsx, sheet_name=sheet)

# xlsx, sheet = parameters('df_ei')
# df_ei = pd.read_excel(xlsx)

xlsx, sheet = parameters('df_estados')
df_estados = pd.read_excel(xlsx)

xlsx, sheet = parameters('df_sg')
df_sg= pd.read_excel(xlsx)

#In general the excel files with just one sheet don't have a good name, so the name was changed to be similar to the name of the excel file
for i, sheet in enumerate(df_xlsx['sheets']):
    if len(sheet)==1:
        new_name = df_xlsx['xlsx_file'][i]
        re_pattern = '(.xlsx?|\d|-|\s)'
        new_name = re.sub(re_pattern,'',new_name)
        df_sheets.sheet_name.loc[df_sheets.sheet_name == sheet[0]] = new_name
    else: pass

for index_1, xlsx_1 in enumerate(df_xlsx['xlsx_file']):
    for index_2, xlsx_2 in enumerate(df_sheets['xlsx_file']):
        if xlsx_1 == xlsx_2 and len(df_xlsx['sheets'][index_1])==1:
            df_xlsx['sheets'][index_1] = df_sheets['sheet_name'][index_2]
        else: pass

df_pr = df_pr[['prop_pk1','prop_1','prop_2','prop_3','prop_4','prop_pk2', 'uf','partido','orgao']]
#----------------------------------------------------------
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time

#Getting out of the Data_Source folder after having read the files
%cd .. 

/home/roberto/github/projeto_controle_financeiro_gesac-11/Data_Source


/home/roberto/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/roberto/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)


Time: This code took 9 seconds to run.
/home/roberto/github/projeto_controle_financeiro_gesac-11


In [139]:
def strip_accents(cln):
   cln = str(cln)
   return ''.join(c for c in unicodedata.normalize('NFD', cln)if unicodedata.category(c) != 'Mn')

def clean_column_name(cln):
    return strip_accents(str(cln).strip('[]()').lower().replace(' ','_'))

def rename_df(df):
    new_cln = [clean_column_name(cln) for cln in df.columns.values]
    old_cln = [cln for cln in df.columns.values]
    dct = dict(zip(old_cln, new_cln))
    df.rename(columns=dct,inplace=True)
    lst_df_clns.append(new_cln)
    return df

#creating a list with all dfs from the data dictionary
dfs = [ x for x in df_sheets.df_name]
cmd = str(dfs).replace("'","") 
exec('dfs = {}'.format(cmd))

lst_df_clns = []
for df in dfs:
    rename_df(df)


df_sheets['columns'] = lst_df_clns 

In [140]:
df_xlsx

,xlsx_file,sheets
0,Controle de Empenhos e NC 2022.xlsx,"[ControleGeral, SintéticoGeral, SintéticoNC, A..."
1,Controle de NC 2022.xlsx,"[SintéticoNC, AnalíticoNC, ÍndiceCorreção]"
2,Emendas2021_resumo.xlsx,Emendas_resumo
3,Orçamentário MCOM 2021 16-11-21 setel.xlsx,"[Adm. Direta - Discricionárias, Emendas Indivi..."
4,Proponente.xlsx,[Sheet1]
5,Proponentes.xlsx,Proponente
6,Relatorio_SISGESAC 1-02-2022.xls,Relatorio_SISGESAC
7,estados.xlsx,estados


In [141]:
df_sheets

,df_name,sheet_name,xlsx_file,columns
0,df_an,AnalíticoNC,Controle de NC 2022.xlsx,"[proponente, nota_de_credito, programa, mes, v..."
1,df_sn,SintéticoNC,Controle de NC 2022.xlsx,"[nc, proponente, programa, empenho, valor_inic..."
2,df_ic,ÍndiceCorreção,Controle de NC 2022.xlsx,"[mes, indice_de_correcao, taxa_selic, observacao]"
3,df_pr,Proponente,Proponentes.xlsx,"[prop_pk1, prop_1, prop_2, prop_3, prop_4, pro..."
4,df_cg,ControleGeral,Controle de Empenhos e NC 2022.xlsx,"[processo, proponente, contrato, nota_de_empen..."
5,df_em,Emendas_resumo,Emendas2021_resumo.xlsx,"[processo, emenda, valor, pontos_livre_(18_mes..."
6,df_estados,estados,estados.xlsx,"[cod, nome, sigla]"
7,df_sg,Relatorio_SISGESAC,Relatorio_SISGESAC 1-02-2022.xls,"[cdgesac, nrpid, situacao, sgunidadefederacao,..."


In [150]:
# criando função 'regexing' p/ subst. nomes abreviados de forma a normalizar as strings
lst_re = [
('PRF ','POLICIA RODOVIARIA FEDERAL '),
('MCOM ', 'MINISTERIO DAS COMUNICACOES '),
('BANCADA ','BANCADA-'),
(r'-D\w ','-'),
(r'(SEN\.?A?/.?|SENADORA?)',''),
(r'(\s?DEPA?\.|\s?DEPUTAD\w?)\s?\s?(FEDERAL)?\s?',''),
(r'MIN/.?\s','MINISTERIO '),
(r'-\s\w*',''),
(r'\(.*\)',''),
]

lamb_str = lambda x:strip_accents(x).upper()
lamb_re = lambda x:regexing(x)
lamb_strip = lambda x: str(x).strip()

df_estados['nome'] = df_estados['nome'].apply(lamb_str)
uf_re = zip(df_estados['nome'], df_estados['sigla'])

for x in list(uf_re):
    lst_re.append(x)

In [143]:
#criando lista dos DataFrames pelo qual o loop passara referente aos proponentes
dfs = (df_sn, df_an,df_cg, df_em, df_sg)

#criando as colunas das primary_keys nos 3 dataframes
for df in dfs:
    df['prop_pk1'] = ''
    df['prop_pk2'] = ''

def regexing(value):
   for string in lst_re:
        value = re.sub(string[0],string[1],value)
   return value

pk2 = []
for index, prop in enumerate(df_pr['prop_pk1']):
    if df_pr['orgao'][index] == "senado":
        pk2.append('SEN. ' + prop)
    elif  df_pr['orgao'][index] == "camara":
        pk2.append('DEP. ' + prop)
    else: 
        pk2.append(prop)
df_pr['prop_pk2'] = pk2

#criando lista dos DataFrames pelo qual o loop passara referente aos proponentes
dfs = (df_sn, df_an,df_cg, df_em, df_sg)

#criando as colunas das primary_keys nos 3 dataframes
for df in dfs:
    df['prop_pk1'] = ''
    df['prop_pk2'] = ''

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 29 seconds to run.


O objetivo inicial do projeto foi limpar os dados relacionados ao os proponentes com a criação de uma coluna nova referente ao nome civil do proponente para que esses valores pudessem ser usados como chave entre diferentes tabelas que serao parametrizadas entre si.

In [154]:
df_sg['proponente'] = df_sg['nosolicitante'] 
df_em['proponente'] = df_em['emenda']

#fazendo o loop sobre a lista acima para aplicar as funções que normalizam a coluna com as chaves dos proponentes
lst_df = [df_sn, df_an, df_cg, df_em, df_sg]

for df in lst_df: 
    df['proponente'] = df['proponente'].apply(lamb_str)
    df['proponente'] = df['proponente'].apply(lamb_re)
    df['proponente'] = df['proponente'].apply(lamb_strip)


lst_pr = (df_pr['prop_pk1'], df_pr['prop_1'],df_pr['prop_2'], df_pr['prop_3'],df_pr['prop_4'])

#criando lista das colunas referente ao nome dos proponentes 
def filter_key(x):
    return (x==lst_pr[0])|(x==lst_pr[1])|(x==lst_pr[2])|(x==lst_pr[3])|(x==lst_pr[4])

for df in lst_df:
    lst_prop_pk1 = []
    lst_prop_pk2 = []
    for index,value in enumerate(df.proponente):
        filt_finding_key =  filter_key(df.proponente.iloc[index])
        result1 = str(df_pr.loc[filt_finding_key].prop_pk1.values).strip("[']")
        result2 = str(df_pr.loc[filt_finding_key].prop_pk2.values).strip("[']")
        lst_prop_pk1.append(result1)
        lst_prop_pk2.append(result2)
    df['prop_pk1'] = lst_prop_pk1
    df['prop_pk2'] = lst_prop_pk2


In [156]:
#cleaning analitic sintetic rows

lst = ['SUBTOTAL','TOTAL','NAN']

for value in lst:
    df_sn = df_sn.loc[df_sn['proponente'] != value ]  
    df_em = df_em.loc[df_em['proponente'] != value ]  
    
df_sn.fillna(0,inplace=True)
df_an.fillna(0,inplace=True)

# df_em.proponente.unique()


In [157]:
start_time = time()

dfs  = [[df_an,'df_an'], [df_sn,'df_sn'],[df_em,'df_em'], [df_sg,'df_sg']]; 

lst_prop_nk= []; lst = []

for df in dfs:
    for index_2, value in enumerate(df[0].loc[df[0].prop_pk1==''].proponente.values):
        if value not in lst:
            prop_name =  df[0].loc[df[0].prop_pk1==''].proponente.values[index_2]
            prop_source = df[1]
            prop_index = (index_2)
            lst_prop_nk.append((prop_name, prop_source, prop_index))
            lst.append(value)
        else:pass

print('DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:')
df_nk = pd.DataFrame()
df_nk['proponente'] = [x for x,y,z in lst_prop_nk]
df_nk['source'] = [y for x,y,z in lst_prop_nk]

print('ROWS:', df_nk.shape[0])
df_nk.head()

DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:
ROWS: 38


,proponente,source,iloc
0,MISAEL VARELLA,df_em,0
1,GILBERTO ABRAMO,df_em,1
2,EUCLYDES PETTERSEN,df_em,2
3,SEN. ANTONIO ANASTASIA,df_em,3
4,JULIO DELGADO,df_em,4


In [173]:
index = 0
df = df_nk['source'].iloc[index]
name = df_nk['proponente'].iloc[index]
exec(f"output = {df}.loc[{df}.proponente == '{name}'][['proponente','prop_pk1','prop_pk2']]")
print(df)
output

df_em


,proponente,prop_pk1,prop_pk2
4,MISAEL VARELLA,,


In [174]:
i = 0

In [189]:
print(f'PROP: {df_nk.proponente.iloc[i]}.')

def writing_in_df_prop():
    prop_name = df_nk['proponente'][i]
    input_regex = input('REGEX: ').upper()
    name_regex = r''+input_regex
    return prop_name, name_regex

prop_name, name_regex = writing_in_df_prop()
#------------------------------------
filter_cl = ['prop_pk1','prop_1','prop_2','prop_3','prop_4','prop_pk2']
filter_name = df_pr['prop_pk1'].str.match(name_regex) == True; 

df_pr.loc[filter_name][filter_cl]

PROP: SEN. ANTONIO ANASTASIA.


,prop_pk1,prop_1,prop_2,prop_3,prop_4,prop_pk2
0,BANCADA-AL,NaN,BANCADA DE ALAGOAS,BANCADA AL,NaN,BANCADA-AL
1,MINISTERIO DAS COMUNICACOES,MCOM (CD),MCOM,MINISTERIO DAS COMUNICACOES,NaN,MINISTERIO DAS COMUNICACOES
2,BANCADA-ES,BANCADA DO ESPIRITO SANTO,Bancada do Espírito Santo,BANCADA ES,NaN,BANCADA-ES
3,BANCADA-MA,BANCADA MA,BANCADA DO MARANHAO,BANCADA DEPUTADOS DO MARANHAO,NaN,BANCADA-MA
4,BANCADA-MG,NaN,BANCADA DE MINAS GERAIS,BANCADA MG,NaN,BANCADA-MG
...,...,...,...,...,...,...
625,JOSE CERQUEIRA DE SANTANA NETO,NaN,NaN,NaN,NaN,DEP. JOSE CERQUEIRA DE SANTANA NETO
626,JOSE SILVA SOARES,NaN,NaN,NaN,NaN,DEP. JOSE SILVA SOARES
627,JOSE VITOR DE RESENDE AGUIAR,NaN,NaN,NaN,NaN,DEP. JOSE VITOR DE RESENDE AGUIAR
628,JOSE CARLOS BECKER DE OLIVEIRA E SILVA,NaN,NaN,NaN,NaN,DEP. JOSE CARLOS BECKER DE OLIVEIRA E SILVA


In [188]:
write_in_df_pr = True
if write_in_df_pr == True:
    n = input('Type the number of the prop: ')
    df_pr.loc[filter_name,f'prop_{n}']= prop_name
    i+=1

df_pr.loc[filter_name]

,prop_pk1,prop_1,prop_2,prop_3,prop_4,prop_pk2,uf,partido,orgao
279,EUCLYDES MARCOS PETTERSEN NETO,EUCLYDES PETTERSEN,NaN,NaN,NaN,DEP. EUCLYDES MARCOS PETTERSEN NETO,MG,PSC,camara


In [190]:
save = False
if save ==True:
    %cd Data_Source
    df_pr.to_excel('Proponente.xlsx')
    %cd ..


In [191]:
dfs = [df_an, df_ic]

for index,df in enumerate(dfs): 
    meses = [mes for mes in df.mes.unique()]
    n_mes = ['01/2021','02/2021','03/2021','04/2021','05/2021','06/2021','07/2021','08/2021','09/2021','10/2021','11/2021','12/2021']
    meses = list(zip(meses, n_mes))

    for index,value in enumerate(meses):
        df['mes'].replace(meses[index][0],meses[index][1],inplace=True)


    df['mes']= pd.to_datetime(df['mes'])
    if (df.mes.dtype == '<M8[ns]') == True:
        print(f'The loop is ok!')
        
    else: print('The loops is broke!')
    

The loop is ok!
The loop is ok!


In [192]:
#df_folders
lst_fd = [
    ('FINANCEIRO','\GESAC\GESAC_2017\ADMPP'),
    ('SISGESAC',None)
    ]

fd = [fd for (pj,fd) in lst_fd]
pj = [pj for (pj,fd) in lst_fd]
df_folders = pd.DataFrame({'project':pj,'folder':fd})

In [193]:
%rm -r Data_Dictionary
%mkdir Data_Dictionary
%mkdir Data_Dictionary/csv/
%mkdir Data_Dictionary/xlsx/
%mkdir Data_Dictionary/csv/Columns/
%mkdir Data_Dictionary/xlsx/Columns/

path = 'Data_Dictionary'

for df in ('df_sheets', 'df_xlsx', 'df_folders'):
    exec(f"{df}.to_csv('{path}/csv/{df}.csv')")
    exec(f"{df}.to_excel('{path}/xlsx/{df}.xlsx')")


path2 = 'Columns'

for i, df  in enumerate(df_sheets['df_name']):
       exec(f'df = {df}')
       lst_cl = []
       for cl in df.columns: lst_cl.append(cl)
       dct = {'columns':lst_cl}
       exec(f"{df_sheets['df_name'][i]}_columns = pd.DataFrame(dct)")
       exec(f"{df_sheets['df_name'][i]}_columns['descricao'] = '' ")
       exec(f"{df_sheets['df_name'][i]}_columns.to_csv('{path}/csv/{path2}/{df_sheets['sheet_name'][i]}_columns.csv')")
       exec(f"{df_sheets['df_name'][i]}_columns.to_excel('{path}/xlsx/{path2}/{df_sheets['sheet_name'][i]}_columns.xlsx')")